## Example 9.7: Find the lowest eigenvalue of the stretched string problem by employing the shooting method described above. 

Start your search at $k=1$ and terminate the search when the eigenvalue is determined within a precision of $10^{-5}$.

Solution: 

We will employ the Numerov method from Example 9.1, which takes as input the initial value of the function and its derivative. We will set $\delta = 1$ to obtain the initial conditions: 

$\phi(x=0) = 0$ and $\phi'(x=0) = 1$.

In [12]:
import numpy as np
import math

# Numerov's algorithm (forward)
# takes as input the initial conditions y(0) and y'(0) as y0 and yp0
# h is the step size, the k-squared term (k2), the S term -- these are FUNCTIONS!
# the initial value of the independent variable x0, and the final value xf
# returns t,y as the solution arrays
def Numerov(k2, S, y0, yp0, h, x0, xf):
    """Returns the solution to a 2nd-order ODEs of the type: y'' + k^2 y = S(x) via the Numerov algorithm"""
    # the number of steps:
    N = int( (xf-x0)/h ) # needs to be an integer
    # get y1 via Taylor series:
    y1 = y0 + h * yp0
    # define the numpy arrays to return
    ya = np.zeros(N+1)
    xa = np.zeros(N+1)
    # set the first two values of the arrays:
    ya[0] = y0
    ya[1] = y1
    xa[0] = x0
    xa[1] = x0 + h
    # integrate via the Numerov algo:
    for n in range(1,N):
        x = x0 + n*h
        xa[n]=x
        h2dt = h**2/12 # appears often so let's just calculate it once!
        ya[n+1] = (2 * (1 - 5*h2dt * k2(x)) * ya[n] - (1 + h2dt *k2(x-h)) * ya[n-1] + h2dt*(S(x+h) + 10 * S(x) + S(x-h)))/((1 + h2dt * k2(x+h) ))    
    xa[N] = xf # set the last x value which is not set in the loop
    return xa,ya

We can use the SciPy bisection function, but let's copy our bisection method from Chapter 6: 

In [8]:
# The bisection algorithm: 
# func should be a function for which we are trying to find the solution, in the form f(x)=0
# xmin and xmax should enclose the root (the function must change signs from xmin to xmax)
# Nmax is the number of evaluations
# prec is the required precision
def bisection(func, xmin, xmax, Nmax, prec): 
    """Function that implements the bisection algorithm for root finding"""
    n = 0 # number of steps taken
    val = 1E99 # the value of the equation, initialize to a large number
    root = math.nan # initialize the root to "not a number"
    while abs(val) > prec and n < Nmax: # loop terminates either when the max number of evals is reached or the precision is reached
        # get the equation values at the edges [xmin, xmax], 
        # and at the bisection point: 
        val = func((xmin+xmax)/2)
        valmax = func(xmax)
        valmin = func(xmin)
        # figure out in which of the two intervals there's a sign change:
        if val * valmax < 0: # sign change between bisection-xmax, set minimum to bisection
            xmin = (xmin+xmax)/2
        elif val * valmin < 0: # sign change between xmin-bisection, set max to bisection
            xmax = (xmin+xmax)/2
        n = n + 1
    if n > Nmax-1:
        print("Warning: maximum number of evaluations exceeded:", Nmax)
    root = (xmin+xmax)/2
    return root, n

We now need to write a function that takes as input the eigenvalue estimate $k$ yields $\phi(1)$.

We will then put that function through our bisection algorithm to get the eigenvalue estimate!

In [51]:
# This function takes as input kest and yields phi(1) using the Numerov method
# It will form the input to our bisection search
def phi1(k):
    """Function that takes as input kest and yields phi(1) using the Numerov method"""
    # Numerov(k2, S, y0, yp0, h, x0, xf) <- function for reference 
    # you can define separate functions for k2 and S, but we can use the lambda method for brevity:
    x,y = Numerov(lambda x: k**2, lambda x: 0, 0, 1, 1E-2, 0, 1)
    return y[-1] # we only need the last element -> phi(1)

Test the function: 

In [52]:
print(phi1(1.)) # this should be non-zero!
print(phi1(np.pi)) # This should be close to zero! pi is an eigenvalue  

0.8414850094813284
-2.029662520774451e-09


Now let's call the bisection function with ```func``` being ```phi1(k)``` and the search starting at 1 and ending at above $\pi$ (where we expect the eigenvalue to be!).

In [53]:
Nmax = 10000
prec = 1E-5
keigen, niter = bisection(phi1, 1, np.pi*1.5, Nmax, prec)
print(keigen)

3.1416365078949573


Is this close to pi? 

In [54]:
print('Fractional difference from pi=', np.abs(keigen-np.pi)/np.pi)

Fractional difference from pi= 1.395925888547985e-05


Indeed, this is pi within the desired precision! 